# Necessary Imports

In [2]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten , Dropout, MaxPooling2D, Activation
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

# Import images

In [3]:
train_image_directory = 'archive/train/'
test_image_directory = 'archive/test/'
size = 224
train_images = []
train_labels = []

good_train_images = os.listdir(train_image_directory + 'good/')

for i,image_name in enumerate(good_train_images):
    image = cv2.imread(train_image_directory + 'good/' + image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
    image = cv2.resize(image, (size, size))  
    image = np.array(image)
    train_images.append(image)
    train_labels.append(0)


In [4]:
bad_train_images = os.listdir(train_image_directory + 'not-good/')
for i,image_name in enumerate(bad_train_images):
    image = cv2.imread(train_image_directory + 'not-good/' + image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (size, size))
    image = np.array(image)
    train_images.append(image)
    train_labels.append(1)

In [5]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)
print(train_images.shape)
print(train_labels.shape)

(300, 224, 224, 3)
(300,)


In [6]:
train_images = train_images.astype('float32') / 255.0
train_images = (train_images - np.mean(train_images)) / np.std(train_images)

In [7]:
test_images = []

test_images_dir = os.listdir(test_image_directory)
for i,image_name in enumerate(test_images_dir):
    image = cv2.imread(test_image_directory + image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (size, size))
    image = np.array(image)
    test_images.append(image)

test_images = np.array(test_images)
print(test_images.shape)

(180, 224, 224, 3)


# Model Implementation

In [8]:
model = Sequential()
input_shape = (size, size, 3)

model.add(Conv2D(filters=96, input_shape=input_shape, kernel_size=(11,11), strides=(4,4), padding='valid', kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid', kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [9]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 activation (Activation)     (None, 54, 54, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 17, 17, 256)       2973952   
                                                                 
 activation_1 (Activation)   (None, 17, 17, 256)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 256)        0         
 2D)                                                    

# Train and validation set split

In [10]:
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.15)

# Model Training

In [11]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
8/8 [==============================] - 15s 2s/step - loss: 2.0016 - accuracy: 0.7882 - val_loss: 1.6869 - val_accuracy: 0.9333
Epoch 2/10
8/8 [==============================] - 12s 2s/step - loss: 1.8116 - accuracy: 0.8157 - val_loss: 1.6041 - val_accuracy: 0.9333
Epoch 3/10
8/8 [==============================] - 12s 1s/step - loss: 1.6915 - accuracy: 0.8157 - val_loss: 1.5187 - val_accuracy: 0.9333
Epoch 4/10
8/8 [==============================] - 13s 2s/step - loss: 1.5743 - accuracy: 0.8157 - val_loss: 1.3401 - val_accuracy: 0.9333
Epoch 5/10
8/8 [==============================] - 12s 2s/step - loss: 1.4845 - accuracy: 0.8157 - val_loss: 1.2078 - val_accuracy: 0.9333
Epoch 6/10
8/8 [==============================] - 12s 2s/step - loss: 1.3952 - accuracy: 0.8157 - val_loss: 1.2913 - val_accuracy: 0.9333
Epoch 7/10
8/8 [==============================] - 13s 2s/step - loss: 1.3001 - accuracy: 0.8157 - val_loss: 1.0865 - val_accuracy: 0.9333
Epoch 8/10
8/8 [==================

# Prediction on Test Images

In [12]:
# predict on test images
predictions = model.predict(test_images)

6/6 [==============================] - 2s 263ms/step


In [13]:
def show_predictions(predictions):
    predictions = 1/(1+np.exp(-predictions))
    predictions = (predictions - predictions.min())/(predictions.max() - predictions.min())
    new_predictions = []
    for i in range(len(predictions)):
        if predictions[i] <= predictions.mean():
            new_predictions.append("test_" + str(i) + " good")
        else:
            new_predictions.append("test_" + str(i) + " not-good")
    return new_predictions

# Print Prediction

In [14]:
show_predictions(predictions)

C:\Users\aniki\AppData\Local\Temp/ipykernel_12088/423631186.py:3: RuntimeWarning: invalid value encountered in true_divide
  predictions = (predictions - predictions.min())/(predictions.max() - predictions.min())


['test_0 not-good',
 'test_1 not-good',
 'test_2 not-good',
 'test_3 not-good',
 'test_4 not-good',
 'test_5 not-good',
 'test_6 not-good',
 'test_7 not-good',
 'test_8 not-good',
 'test_9 not-good',
 'test_10 not-good',
 'test_11 not-good',
 'test_12 not-good',
 'test_13 not-good',
 'test_14 not-good',
 'test_15 not-good',
 'test_16 not-good',
 'test_17 not-good',
 'test_18 not-good',
 'test_19 not-good',
 'test_20 not-good',
 'test_21 not-good',
 'test_22 not-good',
 'test_23 not-good',
 'test_24 not-good',
 'test_25 not-good',
 'test_26 not-good',
 'test_27 not-good',
 'test_28 not-good',
 'test_29 not-good',
 'test_30 not-good',
 'test_31 not-good',
 'test_32 not-good',
 'test_33 not-good',
 'test_34 not-good',
 'test_35 not-good',
 'test_36 not-good',
 'test_37 not-good',
 'test_38 not-good',
 'test_39 not-good',
 'test_40 not-good',
 'test_41 not-good',
 'test_42 not-good',
 'test_43 not-good',
 'test_44 not-good',
 'test_45 not-good',
 'test_46 not-good',
 'test_47 not-good',
 '